In [1]:
import sys
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse

In [2]:
def sigma_ellipse(p, cov, n): 
    eig_vals, eig_vec = np.linalg.eig(cov)
    ang = math.atan2(eig_vec[:,0][1], eig_vec[:,0][0])/math.pi*180
    return Ellipse(p, width=n*math.sqrt(eig_vals[0]),height=n*math.sqrt(eig_vals[1]), angle=ang, fill=False, color="blue", alpha=0.5)

In [3]:
class KalmanFilter: ###kfglobal1kf
    def __init__(self, envmap, init_pose, motion_noise_stds, distance_dev_rate=0.14, direction_dev=0.05):
        c = np.diag([1e+4, 1e+4, 1e+4]) if init_pose is None else np.diag([1e-10, 1e-10, 1e-10]) #globalかどうかで共分散行列を変える
        self.belief = multivariate_normal(mean=init_pose, cov=c) #cov=cに変更
        self.motion_noise_stds = motion_noise_stds
        self.map = envmap 
        self.distance_dev_rate = distance_dev_rate
        self.direction_dev = direction_dev
        
    def observation_update(self, observation): 
        for d in observation:
            z = d[0]
            obs_id = d[1]
            
            ##Hの計算##
            mx, my = self.map.landmarks[obs_id].pos
            mux, muy, mut = self.belief.mean
            q = (mux - mx)**2 + (muy - my)**2
            sqrtq = np.sqrt(q)
            H = np.array([[(mux - mx)/sqrtq, (muy - my)/sqrtq, 0.0],  [(my - muy)/q, (mux - mx)/q, -1.0]])
            
            ##Qの計算##
            hmu = IdealCamera.relative_polar_pos(self.belief.mean, self.map.landmarks[obs_id].pos)
            distance_dev = self.distance_dev_rate*hmu[0]
            Q = np.diag(np.array([distance_dev**2, self.direction_dev**2]))
            
            ##カルマンゲインの計算##
            K = self.belief.cov.dot(H.T).dot(np.linalg.inv(Q + H.dot(self.belief.cov).dot(H.T)))
            
            ##更新##
            self.belief.mean += K.dot(z - hmu)
            self.belief.cov = (np.eye(3) - K.dot(H)).dot(self.belief.cov)
        
    def motion_update(self, nu, omega, time):
        if abs(omega) < 1e-5: omega = 1e-5 #ゼロにすると式が変わるので避ける
        if abs(nu) < 1e-5:         nu = 1e-5

        v = self.motion_noise_stds
        M = np.diag([v["nn"]**2*abs(nu)/time + v["no"]**2*abs(omega)/time, 
                     v["on"]**2*abs(nu)/time + v["oo"]**2*abs(omega)/time])
        
        t = self.belief.mean[2]
        st, ct = math.sin(t), math.cos(t)
        stw, ctw = math.sin(t + omega*time), math.cos(t + omega*time)
        A = np.array([[(stw - st)/omega,    -nu/(omega**2)*(stw - st) + nu/omega*time*ctw],
                                 [(-ctw + ct)/omega, -nu/(omega**2)*(-ctw + ct) + nu/omega*time*stw],
                                 [0,                                time]] )
        
        F = np.diag([1.0, 1.0, 1.0])
        F[0, 2] = nu / omega * (math.cos(t + omega * time) - math.cos(t))
        F[1, 2] = nu / omega * (math.sin(t + omega * time) - math.sin(t))
            
        self.belief.cov = F.dot(self.belief.cov).dot(F.T) + A.dot(M).dot(A.T)
        self.belief.mean = IdealRobot.state_transition(nu, omega, time, self.belief.mean)
        
    def draw(self, ax, elems):
        ##xy平面上の誤差の3シグマ範囲##
        e = sigma_ellipse(self.belief.mean[0:2], self.belief.cov[0:2, 0:2], 3)
        elems.append(ax.add_patch(e))

        ##θ方向の誤差の3シグマ範囲##
        x, y, c = self.belief.mean
        sigma3 = math.sqrt(self.belief.cov[2, 2])*3
        xs = [x + math.cos(c-sigma3), x, x + math.cos(c+sigma3)]
        ys = [y + math.sin(c-sigma3), y, y + math.sin(c+sigma3)]
        elems += ax.plot(xs, ys, color="blue", alpha=0.5)

In [4]:
class KfAgent(Agent): 
    def __init__(self, time_interval, nu, omega, init_pose, envmap, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}):
        super().__init__(nu, omega)
        self.kf = KalmanFilter(envmap, init_pose, motion_noise_stds) 
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None):
        self.kf.motion_update(self.prev_nu, self.prev_omega, self.time_interval) 
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.kf.observation_update(observation) 
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.kf.draw(ax, elems)

In [5]:
def trial(animation): ###kfglobalexec
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation)

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ## ロボットを作る ##
    init_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
    
    a = KfAgent(time_interval, 0.2, 10.0/180*math.pi, None, m)    #初期姿勢をNoneに
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)

    world.draw()
    
    return (r.pose, a.kf.belief.mean)

In [6]:
ok = 0 ###kfglobal1exec
for i in range(1000):
    actual, estm = trial(False)
    diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
    print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
    if diff <= 1.0:
        ok += 1

ok

0 真値: [-0.6658944  -0.56530227  4.68420946] 推定値 [-0.94637343 -0.4566872   4.68218575] 誤差: 0.3007751991108197
1 真値: [-2.16036378 -1.93823065  4.65782071] 推定値 [-1.53817901 -1.70300765  4.454318  ] 誤差: 0.6651644506920666
2 真値: [-2.4538608  -1.50601877  7.69753661] 推定値 [-2.68418959 -1.79526483  7.61158974] 誤差: 0.36974942136915073
3 真値: [1.69506693 1.29088426 2.50873839] 推定値 [1.78153636 1.36607952 8.82898659] 誤差: 0.11459183631134331
4 真値: [ 0.37057763 -3.93099233  7.95101482] 推定値 [ 1.18823179 -1.50321209  6.30770793] 誤差: 2.561771890872117
5 真値: [-5.37370202  0.97251005  6.26870716] 推定値 [-2.70316848  1.13939584  8.16146217] 誤差: 2.6757429382635474
6 真値: [1.957083   1.72001427 3.2110146 ] 推定値 [2.25645772 1.99632285 3.20808879] 誤差: 0.407396187972017
7 真値: [-2.94548175 -3.56609581  4.76155345] 推定値 [-1.25413779 -2.67068706  5.35753564] 誤差: 1.9137401160478533
8 真値: [ 2.47060985 -3.93907977  6.51541724] 推定値 [-1.55501997 -1.57801238  5.43276217] 誤差: 4.6669406095257475
9 真値: [3.87905024 3.97547298 6.

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20 真値: [2.18849452 5.18174508 4.44858068] 推定値 [0.17580073 2.53002369 5.84263186] 誤差: 3.329048301754575
21 真値: [-6.03965078  0.7525997   6.07455017] 推定値 [-2.13478299  1.19436711  8.29799767] 誤差: 3.9297774547670796
22 真値: [-0.42900488 -0.90378236  2.28906518] 推定値 [-0.24719513 -1.03178653  2.23793735] 誤差: 0.2223507485509471
23 真値: [0.97258552 3.21253885 6.37907652] 推定値 [1.29580275 2.51164854 6.68991032] 誤差: 0.7718267981106208
24 真値: [0.98655691 0.82346155 6.26468744] 推定値 [0.90969351 1.19854577 6.12672816] 誤差: 0.38287877346487986
25 真値: [2.80572199 3.09530207 1.86630904] 推定値 [2.78088387 3.09971812 8.44470061] 誤差: 0.025227637851004226
26 真値: [-1.31672875  1.56549776  7.94972226] 推定値 [-1.13778441  1.40028903  7.8998087 ] 誤差: 0.24354671460238678
27 真値: [0.40215199 3.09626948 5.64598386] 推定値 [0.35115761 3.31988478 5.50838234] 誤差: 0.22935611299142608
28 真値: [-3.18832326 -4.02307472  8.36470089] 推定値 [-1.43612451  2.77955203  6.9536603 ] 誤差: 7.024665915261535
29 真値: [5.39887843 5.79134267 3.43201

96 真値: [-3.24906094  1.71612052  7.50594812] 推定値 [-1.00223995  0.59036369  5.21853546] 誤差: 2.513072425144775
97 真値: [-0.28026248  4.84106886  3.8401326 ] 推定値 [-0.44631487  4.53862308  3.82693756] 誤差: 0.34503166091942594
98 真値: [-6.22594793  0.46789257  6.08184428] 推定値 [-1.81456816  0.12352697  7.79069704] 誤差: 4.424800461672781
99 真値: [1.26387573 1.40599908 7.82620041] 推定値 [1.27162803 1.42691693 7.80969946] 誤差: 0.022308179480744185
100 真値: [-4.18986041 -1.36990417  5.22787062] 推定値 [-3.72741469 -0.72227087  5.54812919] 誤差: 0.7957920163257961
101 真値: [-0.31359064 -3.00546031  5.60991566] 推定値 [-0.62938218 -3.74118044  5.93647677] 誤差: 0.8006300001683695
102 真値: [-4.95967412  1.25498649  6.33218788] 推定値 [-3.3392092   2.99082596  3.49682813] 誤差: 2.374667399778454
103 真値: [-1.85126607  3.79981495  3.79247847] 推定値 [-1.54533528  3.68281321  3.66027088] 誤差: 0.32754092208500085
104 真値: [-2.17290064  3.59618486  4.14167186] 推定値 [-2.3814986   3.40443228  4.1413012 ] 誤差: 0.28334106490590033
105 真値: [

171 真値: [ 3.3442367  -1.86741723  8.69675158] 推定値 [ 4.19519342 -1.89872921  2.51235946] 誤差: 0.8515325971797922
172 真値: [1.22973695 2.6047458  4.41113109] 推定値 [1.32680996 2.65573144 4.40558802] 誤差: 0.10964809962674489
173 真値: [3.38741181 4.26566269 6.18411037] 推定値 [1.98829459 2.35697587 2.43781568] 誤差: 2.3665617147331384
174 真値: [-3.77837675  0.21611762  7.63896718] 推定値 [-2.62257706  0.57816855  8.25078486] 誤差: 1.211178684533716
175 真値: [ 0.15307747 -6.20000097  6.78383473] 推定値 [-0.17519912 -0.21201119  4.87710594] 誤差: 5.9969815093684
176 真値: [ 0.46896019 -1.56626955  7.10651221] 推定値 [ 0.57102215 -1.63752702  7.10440587] 誤差: 0.12447598905430729
177 真値: [-1.64377822e+00  6.21671741e-03  8.46036610e+00] 推定値 [-1.70692967  0.33975141  8.52554904] 誤差: 0.3394605966710045
178 真値: [ 2.12903948 -1.2019944   6.38306162] 推定値 [ 2.13434505 -1.09281322  6.3326367 ] 誤差: 0.10931001637253202
179 真値: [ 3.18089185 -3.51951797  5.85369276] 推定値 [ 1.83025331 -1.78884831  1.4322723 ] 誤差: 2.1953227487489073
18

...

772 真値: [-4.65995675  1.66256685  4.17688218] 推定値 [-4.16206236  1.2185589   4.93804126] 誤差: 0.6671145996331906
773 真値: [-0.92274823 -3.68036155  5.98595882] 推定値 [-1.12939822 -3.38514239  5.9391211 ] 誤差: 0.3603589448868029
774 真値: [-1.58059117 -2.60201924  7.76959487] 推定値 [-2.21935502 -2.90308521  7.55365149] 誤差: 0.7061586044473094
775 真値: [ 1.13181763 -1.63948877  6.30964495] 推定値 [ 0.97208343 -1.59313187  6.26585011] 誤差: 0.16632490865384658
776 真値: [ 0.1369216  -1.67999037  4.09540596] 推定値 [-0.19878646 -1.08009738  4.23247425] 誤差: 0.6874383572974216
777 真値: [-1.63628216  0.90715005  5.57563528] 推定値 [-1.67348236  0.60809061  5.57312044] 誤差: 0.30136423519678307
778 真値: [3.97489884 1.34634984 1.62193188] 推定値 [4.05095289 1.74690616 8.12492098] 誤差: 0.40771262292402477
779 真値: [0.23899153 1.35579555 6.83040062] 推定値 [0.75820549 1.20421375 6.94629188] 誤差: 0.5408883172257187
780 真値: [ 3.04717924 -2.45591525  4.83473823] 推定値 [ 0.90147014 -3.82472925  1.75350769] 誤差: 2.5451364080991024
781 真値: [4

847 真値: [1.34481502 4.2348343  4.60813501] 推定値 [1.56691252 4.40542374 4.51728991] 誤差: 0.2800500953077291
848 真値: [4.42700424 1.82882125 9.96988507] 推定値 [ 4.24727075  2.23648204 -2.58546503] 誤差: 0.44552378637358575
849 真値: [-4.15889709  0.65792857  3.57177581] 推定値 [-2.83963029  1.52505368 -1.57066606] 誤差: 1.5787244380894097
850 真値: [-2.80286935  1.05630838  1.72831173] 推定値 [-2.7665202   1.18794926  8.0528848 ] 誤差: 0.13656712787171435
851 真値: [-1.03223028 -2.82602512  5.44969575] 推定値 [-0.94655926 -2.88331186  5.45237548] 誤差: 0.10305966160922996
852 真値: [ 1.21456736 -2.69838213  7.46716775] 推定値 [ 1.04262119 -2.84379984  7.47083137] 誤差: 0.22519279377794812
853 真値: [-5.14496437  4.35069516  5.84804043] 推定値 [-1.25957237  2.98298901  4.06276775] 誤差: 4.119088627358685
854 真値: [-1.89727218  1.8187881   7.68027511] 推定値 [-2.40624396  1.10077548  7.74220265] 誤差: 0.8801104397245866
855 真値: [-1.36892152  2.95238432  6.14143997] 推定値 [-1.48560921  1.66092311  6.38903006] 誤差: 1.2967220424759982
856 真値:

923 真値: [ 2.90922506 -0.70634534  2.86701324] 推定値 [ 2.55280354 -0.79777632  2.82903023] 誤差: 0.36796184787787495
924 真値: [1.738075   5.12410401 2.7222328 ] 推定値 [1.79175405 5.03644053 8.98392982] 誤差: 0.10279263969474663
925 真値: [-3.36377229  5.02908347  7.31548962] 推定値 [-1.71995864  0.31849113  5.12982098] 誤差: 4.9891686218324045
926 真値: [3.47344164 4.22752931 4.85990373] 推定値 [2.44103452 2.02994915 1.53972473] 誤差: 2.4280080373723676
927 真値: [0.15312589 1.46013886 7.00006407] 推定値 [-0.49388857  1.5246464   6.79939168] 誤差: 0.65022222607094
928 真値: [-6.09395319 -2.40437732  5.89155683] 推定値 [-0.2398048  -1.31673115  7.04539896] 誤差: 5.954328473303018
929 真値: [-1.6878378   1.38315257  4.638427  ] 推定値 [-1.42060433  1.59696562  4.5145096 ] 誤差: 0.34224223045290253
930 真値: [-5.25100453 -0.82227585  5.51217498] 推定値 [-3.61862674  0.1292415   6.55697838] 誤差: 1.8894556148767865
931 真値: [-1.27355171 -0.94627991  3.52361043] 推定値 [-1.04032871 -1.03753243  3.45727187] 誤差: 0.25043959359124474
932 真値: [-2.349

998 真値: [ 2.36782659 -3.13705697  4.6783095 ] 推定値 [-1.33658723 -0.33315395  0.33139466] 誤差: 4.645917986255949
999 真値: [ 0.56156184 -1.46675845  3.19201242] 推定値 [ 0.59928331 -1.09460886  3.10928991] 誤差: 0.3740564482412507


529